# 02 Data Cleaning & Feature Engineering – Forex Price Prediction Project

## Objectives
* Load raw OHLCV data for 10 forex pairs from `data/raw/`
* Clean data: 
  - Remove duplicates
  - Handle missing values
  - Ensure proper timestamp formatting
* Engineer key technical indicators for machine learning:
  - SMA50 / SMA200 (trend)
  - RSI (momentum)
  - MACD (trend & momentum)
  - ATR (volatility)
* Save processed CSVs to `data/processed/` for model training.

## Inputs
* Raw CSV files for 10 forex pairs from Notebook 01:
  * data/raw/GBP_USD_forex_data.csv
  * data/raw/EUR_GBP_forex_data.csv
  * data/raw/EUR_USD__forex_data.csv
  * data/raw/AUD_USD_forex_data.csv
  * data/raw/NZD_USD_forex_data.csv
  * data/raw/USD_JPY_forex_data.csv
  * data/raw/USD_SGD_forex_data.csv
  * data/raw/USD_CHF_forex_data.csv
  * data/raw/USD_CAD_forex_data.csv
  * data/raw/GBP_CAD_forex_data.csv 
* Each CSV contains:
- `timestamp`
- `open`
- `high`
- `low`
- `close`
- `volume`

## Outputs
* Cleaned and feature-engineered CSVs saved in:
  * data/processed/GBP_USD_forex_data_processed.csv
  * data/processed/EUR_GBP_forex_data_processed.csv
  * data/processed/EUR_USD__forex_data_processed.csv
  * data/processed/AUD_USD_forex_data_processed.csv
  * data/processed/NZD_USD_forex_data_processed.csv
  * data/processed/USD_JPY_forex_data_processed.csv
  * data/processed/USD_SGD_forex_data_processed.csv
  * data/processed/USD_CHF_forex_data_processed.csv
  * data/processed/USD_CAD_forex_data_processed.csv
  * data/processed/GBP_CAD_forex_data_processed.csv 
* Additional columns for ML:
  - `SMA50`,
  - `SMA200`,
  - `RSI`,
  - `MACD`,
  - `ATR`

## Additional Comments
* This notebook completes the **Data Preparation** stage of CRISP-DM.
* Processed datasets will be used in Notebook 03 for model training.


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/forex-price-predictor/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/forex-price-predictor'

## Section 1: Load Sample Pair
We will start with **EUR/USD** as our sample pair to:
- Load raw OHLCV data from `data/raw/`
- Apply initial cleaning to remove duplicates and handle missing values
- Preview the cleaned data to understand structure

In [4]:
import pandas as pd
from src.data_preprocessing import clean_data

sample_pair = "EUR_USD"
raw_path = f"data/raw/{sample_pair}_forex_data.csv"

# Load and clean sample pair
df_raw = pd.read_csv(raw_path)
df_clean = clean_data(df_raw)

print(f"Loaded {len(df_raw)} rows -> After cleaning: {len(df_clean)} rows")
display(df_clean.head(10))  # Preview first 10 rows


Loaded 8759 rows -> After cleaning: 8759 rows


/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill").fillna(method="bfill")


,timestamp,open,high,low,close,volume
0,2024-03-06 04:00:00+00:00,1.08508,1.08524,1.08495,1.08519,1408
1,2024-03-06 05:00:00+00:00,1.08518,1.08570,1.08516,1.08552,1866
2,2024-03-06 06:00:00+00:00,1.08549,1.08607,1.08549,1.08595,2183
3,2024-03-06 07:00:00+00:00,1.08594,1.08666,1.08586,1.08621,4008
4,2024-03-06 08:00:00+00:00,1.08620,1.08743,1.08592,1.08732,4108
5,2024-03-06 09:00:00+00:00,1.08732,1.08778,1.08694,1.08766,3029
6,2024-03-06 10:00:00+00:00,1.08765,1.08796,1.08734,1.08774,2303
7,2024-03-06 11:00:00+00:00,1.08775,1.08785,1.08716,1.08758,2243
8,2024-03-06 12:00:00+00:00,1.08758,1.08788,1.08674,1.08688,2730
9,2024-03-06 13:00:00+00:00,1.08690,1.08851,1.08685,1.08850,8223


## Section 2: Feature Engineering on Sample Pair
Next, we will:
1. Compute **technical indicators** for ML:
   - SMA50 / SMA200 (trend)
   - RSI (momentum)
   - MACD (trend & momentum)
   - ATR (volatility)
2. Visualize **one comprehensive chart** with:
   - Candlestick + SMA50/SMA200
   - RSI
   - MACD
   - ATR
3. Show a **table of last 10 rows** including indicators


In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.data_preprocessing import add_indicators

# Add technical indicators
df_features = add_indicators(df_clean)

# Create a 4-row subplot
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(
        f"{sample_pair} Candlestick with SMA50 & SMA200",
        "RSI (14-period)",
        "MACD (12,26)",
        "ATR (14-period)"
    )
)

# Row 1: Candlestick + SMA
fig.add_trace(go.Candlestick(
    x=df_features['timestamp'],
    open=df_features['open'],
    high=df_features['high'],
    low=df_features['low'],
    close=df_features['close'],
    name='Candlestick'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_features['timestamp'], y=df_features['SMA50'],
    line=dict(color='blue', width=1.5), name='SMA50'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_features['timestamp'], y=df_features['SMA200'],
    line=dict(color='orange', width=1.5), name='SMA200'
), row=1, col=1)

# Row 2: RSI
fig.add_trace(go.Scatter(
    x=df_features['timestamp'], y=df_features['RSI'],
    line=dict(color='purple', width=1.5), name='RSI'
), row=2, col=1)
fig.add_hline(y=70, line_dash="dot", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dot", line_color="green", row=2, col=1)

# Row 3: MACD
fig.add_trace(go.Scatter(
    x=df_features['timestamp'], y=df_features['MACD'],
    line=dict(color='green', width=1.5), name='MACD'
), row=3, col=1)

# Row 4: ATR
fig.add_trace(go.Scatter(
    x=df_features['timestamp'], y=df_features['ATR'],
    line=dict(color='red', width=1.5), name='ATR'
), row=4, col=1)

fig.update_layout(height=1200, showlegend=True, xaxis_rangeslider_visible=False)
fig.show()

# Display last 10 rows with indicators
display(df_features.tail(10))


,timestamp,open,high,low,close,volume,SMA50,SMA200,RSI,MACD,ATR
8749,2025-07-31 18:00:00+00:00,1.14196,1.14330,1.14157,1.14266,3943,1.148203,1.164779,47.300431,-0.001414,0.002046
8750,2025-07-31 19:00:00+00:00,1.14264,1.14320,1.14122,1.14125,4809,1.147944,1.164652,43.409553,-0.001458,0.002102
8751,2025-07-31 20:00:00+00:00,1.14124,1.14184,1.14096,1.14148,4080,1.147673,1.164527,36.646310,-0.001458,0.001984
8752,2025-07-31 21:00:00+00:00,1.14172,1.14195,1.14170,1.14172,329,1.147408,1.164403,39.910565,-0.001422,0.001835
8753,2025-07-31 22:00:00+00:00,1.14170,1.14229,1.14152,1.14180,1503,1.147153,1.164263,39.679452,-0.001372,0.001769
8754,2025-07-31 23:00:00+00:00,1.14181,1.14263,1.14118,1.14221,3499,1.146899,1.164122,46.121548,-0.001284,0.001740
8755,2025-08-01 00:00:00+00:00,1.14221,1.14289,1.14156,1.14243,5967,1.146648,1.163984,44.452405,-0.001183,0.001686
8756,2025-08-01 01:00:00+00:00,1.14243,1.14288,1.14049,1.14072,6474,1.146361,1.163842,35.655704,-0.001226,0.001751
8757,2025-08-01 02:00:00+00:00,1.14073,1.14150,1.14062,1.14112,5052,1.146049,1.163701,43.297334,-0.001215,0.001626
8758,2025-08-01 03:00:00+00:00,1.14112,1.14223,1.14104,1.14184,3477,1.145757,1.163565,52.921474,-0.001134,0.001539


## Section 3: Process All Pairs and Save ML-Ready CSVs
We will:
1. Loop through all 10 pairs
2. Apply:
   - `clean_data()`
   - `add_indicators()`
   - `add_target()` (binary target for ML)
3. Save processed CSVs to `data/processed/`


In [6]:
from src.data_preprocessing import add_target
import os

PAIRS = [
    "GBP_USD", "EUR_GBP", "EUR_USD", "AUD_USD", "NZD_USD",
    "USD_JPY", "USD_SGD", "USD_CHF", "USD_CAD", "GBP_CAD"
]

processed_dir = "data/processed"
os.makedirs(processed_dir, exist_ok=True)

for pair in PAIRS:
    df = pd.read_csv(f"data/raw/{pair}_forex_data.csv")
    df = clean_data(df)
    df = add_indicators(df)
    df = add_target(df)
    
    save_path = f"{processed_dir}/{pair}_forex_data_processed.csv"
    df.to_csv(save_path, index=False)
    print(f"✅ {pair} processed & saved: {len(df)} rows")


/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



✅ GBP_USD processed & saved: 8759 rows
✅ EUR_GBP processed & saved: 8759 rows


/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



✅ EUR_USD processed & saved: 8759 rows
✅ AUD_USD processed & saved: 8759 rows


/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



✅ NZD_USD processed & saved: 8759 rows
✅ USD_JPY processed & saved: 8759 rows


/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



✅ USD_SGD processed & saved: 8759 rows
✅ USD_CHF processed & saved: 8759 rows


/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/workspaces/forex-price-predictor/src/data_preprocessing.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



✅ USD_CAD processed & saved: 8759 rows
✅ GBP_CAD processed & saved: 8759 rows


## Section 4: Verify Processed Data
We will confirm:
- All 10 processed CSVs exist in `data/processed/`
- Preview the first processed file


In [7]:
import glob

processed_files = glob.glob(f"{processed_dir}/*_processed.csv")
print("Processed files:", processed_files)
print(f"Total processed files: {len(processed_files)}")

# Preview first processed file
if processed_files:
    df_preview = pd.read_csv(processed_files[0])
    display(df_preview.head(10))


Processed files: ['data/processed/USD_SGD_forex_data_processed.csv', 'data/processed/EUR_USD_forex_data_processed.csv', 'data/processed/USD_JPY_forex_data_processed.csv', 'data/processed/USD_CAD_forex_data_processed.csv', 'data/processed/GBP_USD_forex_data_processed.csv', 'data/processed/EUR_GBP_forex_data_processed.csv', 'data/processed/USD_CHF_forex_data_processed.csv', 'data/processed/AUD_USD_forex_data_processed.csv', 'data/processed/GBP_CAD_forex_data_processed.csv', 'data/processed/NZD_USD_forex_data_processed.csv']
Total processed files: 10


,timestamp,open,high,low,close,volume,SMA50,SMA200,RSI,MACD,ATR,target
0,2024-03-06 04:00:00+00:00,1.34257,1.34270,1.34213,1.34215,2967,NaN,NaN,NaN,0.000000,NaN,0
1,2024-03-06 05:00:00+00:00,1.34216,1.34227,1.34172,1.34186,2749,NaN,NaN,NaN,-0.000023,NaN,0
2,2024-03-06 06:00:00+00:00,1.34186,1.34201,1.34162,1.34164,3539,NaN,NaN,NaN,-0.000059,NaN,0
3,2024-03-06 07:00:00+00:00,1.34164,1.34167,1.34080,1.34116,3538,NaN,NaN,NaN,-0.000124,NaN,0
4,2024-03-06 08:00:00+00:00,1.34118,1.34166,1.34088,1.34116,3940,NaN,NaN,NaN,-0.000174,NaN,0
5,2024-03-06 09:00:00+00:00,1.34119,1.34145,1.34086,1.34086,2704,NaN,NaN,NaN,-0.000235,NaN,1
6,2024-03-06 10:00:00+00:00,1.34088,1.34154,1.34082,1.34140,3972,NaN,NaN,NaN,-0.000237,NaN,0
7,2024-03-06 11:00:00+00:00,1.34140,1.34145,1.34096,1.34106,2892,NaN,NaN,NaN,-0.000263,NaN,0
8,2024-03-06 12:00:00+00:00,1.34105,1.34109,1.34066,1.34102,2943,NaN,NaN,NaN,-0.000283,NaN,0
9,2024-03-06 13:00:00+00:00,1.34102,1.34120,1.34002,1.34005,5889,NaN,NaN,NaN,-0.000374,NaN,0


---

# Push processed data and notebook updates to GitHub

### 1. Check Git Status

In [8]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   jupyter_notebooks/02_data_cleaning_feature_engineering.ipynb
	modified:   src/data_preprocessing.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/processed/

no changes added to commit (use "git add" and/or "git commit -a")


### 2 . Stage All New/Updated Files

In [9]:
!git add data/processed/*.csv notebooks/02_data_cleaning_feature_engineering.ipynb

fatal: pathspec 'notebooks/02_data_cleaning_feature_engineering.ipynb' did not match any files


### 3. Commit With A Clear Message

In [10]:
!git commit -m "Add processed forex data with technical indicators and target (Notebook 02 complete)"

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   jupyter_notebooks/02_data_cleaning_feature_engineering.ipynb
	modified:   src/data_preprocessing.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/processed/

no changes added to commit (use "git add" and/or "git commit -a")


### 4. Push To The Main Branch

In [11]:
!git push origin main

Everything up-to-date
